In [ ]:
using Plots
using LinearAlgebra
using Krylov
using Printf
using LaTeXStrings
using BenchmarkTools
using DifferentialEquations
include("poisson2d.jl")

In [ ]:
default(lw=2, markersize=6,
    xtickfont=font(12), ytickfont=font(12),
    guidefont=font(14), legendfont=font(12), titlefont=font(12))

# Heat Equation Example
Solve
$$
u_t = \Delta u
$$
over $(0,L_x)\times (0,L_y)$ with zero Dirichlet boundary conditions.

In [ ]:
Lx = 5;
Ly = 5;
nx = 99; # (nx-1)×(ny-1) interior points
ny = 99;

x = LinRange(0, Lx, nx + 2)
y = LinRange(0, Ly, ny + 2)

@show Δx = x[2] - x[1];
@show Δy = y[2] - y[1];


xy = [[x_, y_] for x_ in x, y_ in y]; # mesh including boundary points
xy_int = [[x_, y_] for x_ in x[2:end-1], y_ in y[2:end-1]]; # interior mesh points

In [ ]:
u0= [Float64((1.5<x_[1]<3.5)*(1.5<x_[2]<3.5)) for x_ in xy_int]
U0 = u0[:];
surface(x[2:end-1], y[2:end-1], u0', label="")
xlabel!(L"$x$")
ylabel!(L"$y$")

In [ ]:
function fheat2d_dirichlet!(dU, U, p, t)
    Δx = p[1];
    Δy = p[2];
    nx = Int(p[3]);
    ny = Int(p[4]);
    n = length(U); # = nx * ny
    
    @. dU = 0;

    for k = 1:n
        j, i = fldmod1(k, nx);
        # corners, then edges, then interior
        if (i == 1 && j == 1)
            dU[k] = (-2 * U[i + nx * (j-1)] + U[i+1 + nx *(j-1)])/Δx^2 + (-2 * U[i + nx * (j-1)] + U[i + nx *(j)])/Δy^2;
        elseif (i == 1 && j == ny)
            dU[k] = (-2 * U[i + nx * (j-1)] + U[i+1 + nx *(j-1)])/Δx^2 + (-2 * U[i + nx * (j-1)] + U[i + nx *(j-2)])/Δy^2;
        elseif (i == nx && j == 1)
            dU[k] = (-2 * U[i + nx * (j-1)] + U[i-1 + nx *(j-1)])/Δx^2 + (-2 * U[i + nx * (j-1)] + U[i + nx *(j)])/Δy^2;
        elseif (i == nx && j == ny)
            dU[k] = (-2 * U[i + nx * (j-1)] + U[i-1 + nx *(j-1)])/Δx^2 + (-2 * U[i + nx * (j-1)] + U[i + nx *(j-2)])/Δy^2;
        elseif (i == 1 && j > 1 && j < ny)
            # right edge of domain, (1<j<ny)
            dU[k] = (-2 * U[i + nx * (j-1)] + U[i+1 + nx *(j-1)])/Δx^2 + (U[i + nx *(j-2)]-2 * U[i + nx * (j-1)] + U[i + nx *(j)])/Δy^2;
        elseif (i == nx && j > 1 && j < ny)
            # left edge of domain, (1<j<ny)
            dU[k] = (-2 * U[i + nx * (j-1)] + U[i-1 + nx *(j-1)])/Δx^2 + (U[i + nx *(j-2)]-2 * U[i + nx * (j-1)] + U[i + nx *(j)])/Δy^2;
        elseif (i > 1 && i < nx && j == 1)
            # bottom edge of domain, (1<i<nx)
            dU[k] = (U[i-1 + nx *(j-1)]-2 * U[i + nx * (j-1)] + U[i+1 + nx *(j-1)])/Δx^2 + (-2 * U[i + nx * (j-1)] + U[i + nx *(j)])/Δy^2;
        elseif (i > 1 && i < nx && j == ny)
            # top edge of domain, (1<i<nx)
            dU[k] = (U[i-1 + nx *(j-1)]-2 * U[i + nx * (j-1)] + U[i+1 + nx *(j-1)])/Δx^2 + (-2 * U[i + nx * (j-1)] + U[i + nx *(j-2)])/Δy^2;
        else
            # interior of domain
            dU[k] = (U[i-1 + nx *(j-1)]-2 * U[i + nx * (j-1)] + U[i+1 + nx *(j-1)])/Δx^2 + (U[i + nx *(j-2)]-2 * U[i + nx * (j-1)] + U[i + nx *(j)])/Δy^2;
        end
    end
    dU
end

In [ ]:
p = [Δx, Δy, nx, ny];
tspan = (0.0, 1.0);
prob_dirichlet = ODEProblem(fheat2d_dirichlet!, U0, tspan, p);
sol = solve(prob_dirichlet, saveat=0.05);

In [ ]:
anim = @animate for (t, U) in zip(sol.t, sol.u)
    surface(x[2:end-1], y[2:end-1], reshape(U,nx,ny)', label="", color=:viridis)
    title!(@sprintf("t = %.2f", t))
    xlabel!(L"$x$")
    ylabel!(L"$y$")
    zlims!(0, 1.5)
end

In [ ]:
gif(anim, fps=3)


In [ ]:
function fheatsource2d_dirichlet!(dU, U, p, t)
    Δx = p[1]
    Δy = p[2]
    nx = Int(p[3])
    ny = Int(p[4])
    n = length(U)

    @. dU = 0

    for k = 1:n
        j, i = fldmod1(k, nx);
        x = i * Δx;
        y = j * Δy;
        # corners, then edges, then interior
        if (i == 1 && j == 1)
            dU[k] = (-2 * U[i+nx*(j-1)] + U[i+1+nx*(j-1)]) / Δx^2 + (-2 * U[i+nx*(j-1)] + U[i+nx*(j)]) / Δy^2
        elseif (i == 1 && j == ny)
            dU[k] = (-2 * U[i+nx*(j-1)] + U[i+1+nx*(j-1)]) / Δx^2 + (-2 * U[i+nx*(j-1)] + U[i+nx*(j-2)]) / Δy^2
        elseif (i == nx && j == 1)
            dU[k] = (-2 * U[i+nx*(j-1)] + U[i-1+nx*(j-1)]) / Δx^2 + (-2 * U[i+nx*(j-1)] + U[i+nx*(j)]) / Δy^2
        elseif (i == nx && j == ny)
            dU[k] = (-2 * U[i+nx*(j-1)] + U[i-1+nx*(j-1)]) / Δx^2 + (-2 * U[i+nx*(j-1)] + U[i+nx*(j-2)]) / Δy^2
        elseif (i == 1 && j > 1 && j < ny)
            # right edge of domain, (1<j<ny)
            dU[k] = (-2 * U[i+nx*(j-1)] + U[i+1+nx*(j-1)]) / Δx^2 + (U[i+nx*(j-2)] - 2 * U[i+nx*(j-1)] + U[i+nx*(j)]) / Δy^2
        elseif (i == nx && j > 1 && j < ny)
            # left edge of domain, (1<j<ny)
            dU[k] = (-2 * U[i+nx*(j-1)] + U[i-1+nx*(j-1)]) / Δx^2 + (U[i+nx*(j-2)] - 2 * U[i+nx*(j-1)] + U[i+nx*(j)]) / Δy^2
        elseif (i > 1 && i < nx && j == 1)
            # bottom edge of domain, (1<i<nx)
            dU[k] = (U[i-1+nx*(j-1)] - 2 * U[i+nx*(j-1)] + U[i+1+nx*(j-1)]) / Δx^2 + (-2 * U[i+nx*(j-1)] + U[i+nx*(j)]) / Δy^2
        elseif (i > 1 && i < nx && j == ny)
            # top edge of domain, (1<i<nx)
            dU[k] = (U[i-1+nx*(j-1)] - 2 * U[i+nx*(j-1)] + U[i+1+nx*(j-1)]) / Δx^2 + (-2 * U[i+nx*(j-1)] + U[i+nx*(j-2)]) / Δy^2
        else
            # interior of domain
            dU[k] = (U[i-1+nx*(j-1)] - 2 * U[i+nx*(j-1)] + U[i+1+nx*(j-1)]) / Δx^2 + (U[i+nx*(j-2)] - 2 * U[i+nx*(j-1)] + U[i+nx*(j)]) / Δy^2
        end
        dU[k] += Float64((2.5 < x < 3.) * (2.5 < y < 3.));
    end
    dU
end

In [ ]:
U0 = zeros(nx*ny);
p = [Δx, Δy, nx, ny];
tspan = (0.0, 10.0);
source_prob_dirichlet = ODEProblem(fheatsource2d_dirichlet!, U0, tspan, p);

In [ ]:
sol = solve(source_prob_dirichlet, saveat=0.1)

In [ ]:
anim = @animate for (t, U) in zip(sol.t, sol.u)
    surface(x[2:end-1], y[2:end-1], reshape(U, nx, ny)', label="")
    title!(@sprintf("t = %.2f", t))
    xlabel!(L"$x$")
    ylabel!(L"$y$")
    zlims!(0, .1)
end

In [ ]:
gif(anim, fps=6)


In [ ]:
function freactiondiffusion2d_dirichlet!(dU, U, p, t)
    Δx = p[1]
    Δy = p[2]
    nx = Int(p[3])
    ny = Int(p[4])
    n = length(U)

    @. dU = 0

    for k = 1:n
        j, i = fldmod1(k, nx);
        # corners, then edges, then interior
        if (i == 1 && j == 1)
            dU[k] = (-2 * U[i+nx*(j-1)] + U[i+1+nx*(j-1)]) / Δx^2 + (-2 * U[i+nx*(j-1)] + U[i+nx*(j)]) / Δy^2
        elseif (i == 1 && j == ny)
            dU[k] = (-2 * U[i+nx*(j-1)] + U[i+1+nx*(j-1)]) / Δx^2 + (-2 * U[i+nx*(j-1)] + U[i+nx*(j-2)]) / Δy^2
        elseif (i == nx && j == 1)
            dU[k] = (-2 * U[i+nx*(j-1)] + U[i-1+nx*(j-1)]) / Δx^2 + (-2 * U[i+nx*(j-1)] + U[i+nx*(j)]) / Δy^2
        elseif (i == nx && j == ny)
            dU[k] = (-2 * U[i+nx*(j-1)] + U[i-1+nx*(j-1)]) / Δx^2 + (-2 * U[i+nx*(j-1)] + U[i+nx*(j-2)]) / Δy^2
        elseif (i == 1 && j > 1 && j < ny)
            # right edge of domain, (1<j<ny)
            dU[k] = (-2 * U[i+nx*(j-1)] + U[i+1+nx*(j-1)]) / Δx^2 + (U[i+nx*(j-2)] - 2 * U[i+nx*(j-1)] + U[i+nx*(j)]) / Δy^2
        elseif (i == nx && j > 1 && j < ny)
            # left edge of domain, (1<j<ny)
            dU[k] = (-2 * U[i+nx*(j-1)] + U[i-1+nx*(j-1)]) / Δx^2 + (U[i+nx*(j-2)] - 2 * U[i+nx*(j-1)] + U[i+nx*(j)]) / Δy^2
        elseif (i > 1 && i < nx && j == 1)
            # bottom edge of domain, (1<i<nx)
            dU[k] = (U[i-1+nx*(j-1)] - 2 * U[i+nx*(j-1)] + U[i+1+nx*(j-1)]) / Δx^2 + (-2 * U[i+nx*(j-1)] + U[i+nx*(j)]) / Δy^2
        elseif (i > 1 && i < nx && j == ny)
            # top edge of domain, (1<i<nx)
            dU[k] = (U[i-1+nx*(j-1)] - 2 * U[i+nx*(j-1)] + U[i+1+nx*(j-1)]) / Δx^2 + (-2 * U[i+nx*(j-1)] + U[i+nx*(j-2)]) / Δy^2
        else
            # interior of domain
            dU[k] = (U[i-1+nx*(j-1)] - 2 * U[i+nx*(j-1)] + U[i+1+nx*(j-1)]) / Δx^2 + (U[i+nx*(j-2)] - 2 * U[i+nx*(j-1)] + U[i+nx*(j)]) / Δy^2
        end
        dU[k] += U[k]^2 * (1-U[k]) # nonlinearity
    end
    dU
end

In [ ]:
u0= [Float64((1.5<x_[1]<3.5)*(1.5<x_[2]<3.5)) for x_ in xy_int];
U0 = u0[:];
p = [Δx, Δy, nx, ny];
tspan = (0.0, 1.0);
reactiondiffuion_prob_dirichlet = ODEProblem(freactiondiffusion2d_dirichlet!, U0, tspan, p);

In [ ]:
sol = solve(reactiondiffuion_prob_dirichlet, saveat=0.1);

In [ ]:
anim = @animate for (t, U) in zip(sol.t, sol.u)
    surface(x[2:end-1], y[2:end-1], reshape(U, nx, ny)', label="")
    title!(@sprintf("t = %.2f", t))
    xlabel!(L"$x$")
    ylabel!(L"$y$")
    zlims!(0, 1)
end

In [ ]:
gif(anim, fps=6)
